In [1]:
import sagemaker
import boto3
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
import pandas as pd
import numpy as np

In [3]:
from io import StringIO

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from sagemaker.model import Model
from sagemaker.transformer import Transformer

In [6]:
# Khởi tạo session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [7]:
# tag_name_speed = "DWA_ACTUAL_MOTOR_SPEED"
# # tag_name_speed = "DWB_ACTUAL_MOTOR_SPEED"
# # tag_name_speed = "DWC_ACTUAL_MOTOR_SPEED"

In [8]:
# cluster = 2

In [9]:
# unit = "DegC"

In [10]:
# # tag_name = "DW_AMBIENT_TEMP"

# tag_name = "DW_GEARBOX1_LO_PRESS"

In [65]:
# # DegC
# tag_name_units = [
#     'DWC_RTD1_TEMP', 
#     'DWB_EXHAUST_TEMP', 
#     'DW_GEARBOX1_LO_TEMP',
#     'DWB_RTD2_TEMP', 
#     'DW_GEARBOX2_LO_TEMP', 
#     'DWA_RTD2_TEMP',
#     # 'TD_RTD2_TEMP', 
#     'DWA_RTD3_TEMP', 
#     # 'TD_RTD3_TEMP', 
#     'DWB_RTD3_TEMP',
#     'DWA_RTD1_TEMP', 
#     'DWA_EXHAUST_TEMP', 
#     'DW_GEARBOX2_LO_PRESS',
#     # 'TD_RTD1_TEMP', 
#     'DWC_RTD3_TEMP', 
#     'DW_GEARBOX1_LO_PRESS',
#     'DWC_RTD2_TEMP', 
#     'DW_AMBIENT_TEMP', 
#     'DWB_RTD1_TEMP',
#     'DWC_EXHAUST_TEMP'
# ]
# tag_name_units

In [66]:
# filename = ""

In [22]:
# s3_input_train = f"s3://s3-assetcare-bucket/features_store2/training/ready/{tag_name_speed}_cluster{cluster}_{unit}_{tag_name}"
s3_input_train = "s3://swo-ngoctran-public/mlops/raw_data/2024-01"
s3_input_train

's3://swo-ngoctran-public/mlops/raw_data/2024-01'

In [23]:
# output_path = f"s3://s3-assetcare-bucket/features_store2/result/{tag_name_speed}_cluster{cluster}_{unit}_{tag_name}/"
output_path = "s3://swo-ngoctran-public/mlops/raw_data/2024-01/"

In [24]:
# filename = f"{tag_name_speed}_cluster{str(cluster)}_{unit}_{tag_name}_origin.csv"
filename = "test_training_part_0000.csv"
filename

'test_training_part_0000.csv'

In [25]:
s3_input_train + "/" + filename

's3://swo-ngoctran-public/mlops/raw_data/2024-01/test_training_part_0000.csv'

In [26]:
# df = pd.read_csv(s3_input_train + "/" + filename, nrows=1, header=None)
# num_columns = df.shape[1]
# num_columns

In [27]:
df = pd.read_csv(s3_input_train + "/" + filename, nrows=1, header=None)
num_columns = df.shape[1]
num_columns

8

In [33]:
df.head()

,0,1,2,3,4,5,6,7
0,8727.927,8728.56,8728.56,8728.877,8729.511,8729.511,8730.144,0.0


In [31]:
def training_model(s3_input_train, output_path, feature_dim, num_samples_per_tree, num_trees):
    # Lấy container mới nhất của Random Cut Forest trên SageMaker
    rcf_container = sagemaker.image_uris.retrieve("randomcutforest", region)

    # Cấu hình SageMaker Estimator
    rcf_estimator = Estimator(
        image_uri=rcf_container,
        role=role,
        instance_count=1,  # Nếu dữ liệu lớn, có thể tăng lên 2-3 instance
        instance_type="ml.m5.xlarge",
        output_path=output_path,
        sagemaker_session=sagemaker_session,
        enable_network_isolation=True,
    )

    # **Thêm thông số training: feature_dim = 4**
    rcf_estimator.set_hyperparameters(
        # shingle_size=10,
        feature_dim=feature_dim-1,  # Số lượng cột đặc trưng
        num_samples_per_tree=num_samples_per_tree,  
        num_trees=num_trees  
    )

    # Định nghĩa dữ liệu đầu vào với "ShardedByS3Key"
    train_input = TrainingInput(
        s3_input_train,
        distribution="ShardedByS3Key",  # Chia dữ liệu theo file S3
        content_type="text/csv",
        input_mode="Pipe",  # Dùng Pipe Mode để stream data từ S3
    )

    # Train model
    rcf_estimator.fit({"train": train_input}, wait=True)
    return rcf_estimator

In [34]:
s3_input_train

's3://swo-ngoctran-public/mlops/raw_data/2024-01'

In [35]:
rcf_estimator = training_model(
    s3_input_train=s3_input_train, 
    output_path=output_path, 
    feature_dim=num_columns, 
    num_samples_per_tree=1024, 
    num_trees=300
)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: randomcutforest-2025-06-16-04-01-49-756


2025-06-16 04:01:49 Starting - Starting the training job...
2025-06-16 04:02:19 Starting - Preparing the instances for training...
2025-06-16 04:02:40 Downloading - Downloading input data...
2025-06-16 04:03:00 Downloading - Downloading the training image........Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:495: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if cons['type'] is 'ineq':
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:743: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(self.X_min) is not 0:
[06/16/2025 04:04:38 INFO 140657515337536] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-conf.json: {'num_sa

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 rcf_estimator = training_model(                                                              │
│   2 │   s3_input_train=s3_input_train,                                                           │
│   3 │   output_path=output_path,                                                                 │
│   4 │   feature_dim=num_columns,                                                                 │
│                                                                                                  │
│ in training_model:33                                                                             │
│                                                                                                  │
│   30 │   )                                                                                       │
│   31 │                                                                                           │
│   32 │   # Train model                                                                           │
│ ❱ 33 │   rcf_estimator.fit({"train": train_input}, wait=True)                                    │
│   34 │   return rcf_estimator                                                                    │
│   35                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py:167 in wrapper  │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py:138 in wrapper  │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                            

In [ ]:
rcf_predictor = rcf_estimator.deploy(
    initial_instance_count=1,  # instance runs endpoint
    instance_type="ml.m5.large"  # instance type
)

In [30]:
output_path

's3://swo-ngoctran-public/mlops/raw_data/2024-01/'

In [56]:
def get_model_path(tag_name_speed, cluster, unit, tag_name):
    s3 = boto3.client("s3")

    bucket = "s3-assetcare-bucket"
    prefix = f"features_store2/result/{tag_name_speed}_cluster{cluster}_{unit}_{tag_name}/"
    
    # List all "folders" (S3 prefixes) trong thư mục đó
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, Delimiter="/")
    
    # Lọc ra folder có tên bắt đầu bằng "randomcutforest-"
    candidates = [
        cp['Prefix'] for cp in response.get('CommonPrefixes', [])
        if cp['Prefix'].startswith(prefix + "randomcutforest-")
    ]
    return "s3://" + bucket + "/" + candidates[0] + "output/model.tar.gz"

In [57]:
# s3://s3-assetcare-bucket/features_store2/result/DWA_ACTUAL_MOTOR_SPEED_cluster2_DegC_DW_AMBIENT_TEMP/randomcutforest-2025-03-28-04-57-48-276/output/model.tar.gz

In [74]:
model_path = get_model_path(tag_name_speed=tag_name_speed, cluster=cluster, unit=unit, tag_name=tag_name)
model_path

's3://s3-assetcare-bucket/features_store2/result/DWA_ACTUAL_MOTOR_SPEED_cluster2_DegC_DW_GEARBOX1_LO_PRESS/randomcutforest-2025-03-28-06-19-04-447/output/model.tar.gz'

In [59]:
def get_prediction(model_path, feature_s3_path, output_path):
    # Lấy container mới nhất của Random Cut Forest trên SageMaker
    rcf_container = sagemaker.image_uris.retrieve("randomcutforest", region)

    # Load model từ S3
    rcf_model = Model(
        image_uri=rcf_container,  # Dùng container SageMaker Random Cut Forest
        model_data=model_path,  # Đường dẫn model trên S3
        role=role,
        sagemaker_session=sagemaker_session
    )

    nr_of_files = 1
    # predict anomaly scores
    for idx in range(nr_of_files):
        # test input
        print(idx)
        print(feature_s3_path)
    
        # test output
        # output_path = f"s3://s3-assetcare-bucket/features_store2/DWA/quicktrain/results_set/"
        print(output_path)
    
        transformer = rcf_model.transformer(
            instance_count=1,
            instance_type="ml.m5.xlarge",
            output_path=output_path,
            assemble_with="Line",
            accept="text/csv"
        )
        
        # Chạy dự đoán trên dữ liệu S3
        transformer.transform(feature_s3_path, content_type="text/csv", split_type="Line", wait=True)

In [75]:
feature_s3_path = f"s3://s3-assetcare-bucket/features_store2/training/get_scores/{tag_name_speed}_cluster{cluster}_{unit}_{tag_name}/"

output_path = f"s3://s3-assetcare-bucket/features_store2/training/results/"

In [76]:
get_prediction(model_path=model_path, feature_s3_path=feature_s3_path, output_path=output_path)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: randomcutforest-2025-03-28-06-23-23-017


0
s3://s3-assetcare-bucket/features_store2/training/get_scores/DWA_ACTUAL_MOTOR_SPEED_cluster2_DegC_DW_GEARBOX1_LO_PRESS/
s3://s3-assetcare-bucket/features_store2/training/results/


INFO:sagemaker:Creating transform job with name: randomcutforest-2025-03-28-06-23-23-673


...................................Docker entrypoint called with argument(s): serve
Running default environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:495: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if cons['type'] is 'ineq':
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:743: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(self.X_min) is not 0:
[03/28/2025 06:29:17 INFO 139722077046592] loaded entry point class algorithm.serve.server_config:config_api
[03/28/2025 06:29:17 INFO 139722077046592] loading entry points
[03/28/2025 06:29:17 INFO 139722077046592] Loaded iterator creator application/x-recordio-protobuf for content type ('application/x-recordio-protobuf', '1.0')
[03/28/2025 06:29:17 INFO 139722077046592] loaded request ite